# Commands to set Googe Cloud instances, replicate instances. load data to instance and more

### Connect to Cloud

- Set an instance at VM instances
- Create SSH: 
    - Run ssh-keygen and save with specific name(Example google_machine_key) 
        - Note that you can speficy user name as such: ssh-keygen -t rsa -f ~/.ssh/shimkin_cloud_key -C shimkin

    - cat google_machine_key.pub  and paste it to SSH in Cloud (In Google Cloud ->VM instances ->Edit -> Show edit SSH -> paste to empty place 

    - h -i google_machine_key 35.237.77.116 -v 

    - To logto your previous session run:  
        tmux a -t 0      (Install with sudo apt install tmux) 

Note: To login to specific user add its name:
ssh -i google_machine_key  shimkin@35.196.175.183   -v

Read more on ssh keys for cloud: https://cloud.google.com/compute/docs/instances/adding-removing-ssh-keys#createsshkeys

### Create Snapshot of instance
    - Go to computer Engine - >Snapshots and create a snapshot from the instance
    - Interesting to know - Snapshots are extending from each other, meaning that snapshot2 saves only the changes from snapshot 1 if they are from the same instance 
    - When creating new instance you can set from snapshot.
    - If you changed the size of the SSD memory you need ot mount the edition memory space
            - 


## Load data to VM Instance

Check out this link:
https://cloud.google.com/compute/docs/instances/transfer-files#transfergcloud

There are multiple ways to load data to VM. Some need access to cloud storage. methods:
    - Using the scripts I created to access Dropbox and Google Drive acout - These scripts are Python scrpits that access a the accounts and download data
    - Using google Storage:
        - Go to Google Cloud Platform - >Storage. and Create a Bucket
        - Upload a file to the Bucket
            - Use the Upload files/folder buttons
            - Using gsutils
        - Download the file to you VM  using gsutils
    - Using gcloud compute 
        - Essentially accessing the VM straight from you local machine and vise verse

### gsutil

Link:  https://cloud.google.com/storage/docs/gsutil/commands/cp

This is a very useful tool for controlling the Cloud stuff. I haven't gotten into it enough and you should check he link for more functionalities.

#### gsutil cp
Copying or dowloading files from a machine (local computer or VM in cloud) storage cloud.
Usages:

Downloading a file
- gsutil cp data_file gs://my-bucket  #Will copy data_file to the storage
- gsutil cp gs://my-bucket/data_file data_file  #Will download data_file from the storage

Downloading a directory:
- gsutil -m cp -r dir gs://my-bucket

### gcloud
Methods to connect VM to local machine and generally conrtol the VM. Need to dive deeper into it
Links:
- Methods for transfer to transfer data 
https://cloud.google.com/compute/docs/instances/transfer-files#transfergcloud
- scp command: 
https://cloud.google.com/sdk/gcloud/reference/compute/scp

Usage example:
- gcloud compute scp temp/my_transfer.txt  instance-6:~/temp --zone us-east1-d # Copy a file from computer to VM. This copies the file temp/my_transfer.txt to the VM instacne 'instance-6 at to name temp. Also the Zone need to be speficied (Which you can check out at the VM GUI
- gcloud compute scp --recurse [INSTANCE_NAME]:[REMOTE_DIR] [LOCAL_DIR]  #To copy a directory
    - gcloud compute scp --recurse example-instance:~/narnia ~/wardrobe 
    
    
Note:
- Sometimes the cloud sets a different user and data is transfered to it. In this case you can basically goto the user you what to write or read from:
    - gcloud compute scp  instance-6:~/../shimkin/my_file.txt my_file.txt --zone us-east1-d   #From Blink user I go back and fetch data out of shimkin user





### File Browser
Use will be able to open a file browers presention the VM instance
Link, again at:    https://cloud.google.com/compute/docs/instances/transfer-files#transfergcloud

Steps:
- Obtain an ssh key (As you did before)
- echo "IdentityFile ~/.ssh/my-ssh-key" >> ~/.ssh/config   #Add a line (with your key name) to ssh config
#Open file browser and establish connection to instance via sftp:
sftp://shimkin@35.196.175.183
(I was unable to make it work for now)


### scp command (Without gcloud)
Very easy method:
- scp -i ~/.ssh/my-ssh-key [LOCAL_FILE_PATH] [USERNAME]@[IP_ADDRESS]:~   #To upload
- scp -i ~/.ssh/my-ssh-key [USERNAME]@[IP_ADDRESS]:[REMOTE_FILE_PATH] [LOCAL_FILE_PATH] #To download 


Example:
scp -i  ~/.ssh/shimkin_cloud_key temp_file.txt shimkin@35.196.175.183:~

Note: Not sure yet how to load a hole directory

# Storage

[Here](https://cloud.google.com/persistent-disk/) you can find aditional information on persistent disk. Including pricing:
<img src="figs/persistent_disk_pricing.png" alt="drawing" width="400px"/>

###  Adding or Resizing Persistent Disks 
This is essential when using cloud service. By adding and removing disks you can:
1. Resize a disk on VM that you are currently using when you run out of space, thus not requiring to start with a large disk
2. Resize a Snapshop or image of a disk when you would like to have another large intance

Sources:
1. [Official Google Guild to Storage](https://cloud.google.com/compute/docs/disks/add-persistent-disk) - Great walk through the process with explanations

### Adding new Disk to Instance

1. In the instance go to Edit-> Aditional Disk -> Add Item  and add the amount you want. Remember your DEVICE_ID (The name you gave the instance) 
<br>In the VM shell:
2. Show the partitions. Your new disk will apear there:
    - sudo lsblk  
3. Delete and format the disk with mkfs command:
    - sudo mkfs.ext4 -m 0 -F -E lazy_itable_init=0,lazy_journal_init=0,discard /dev/[DEVICE_ID_FROMLlsblk] 
  <br>Note - Since addition disk doesn't require space for OS we set -m 0 to use all available space
4. Mount to disk to a location you would like:
   - sudo mkdir -p /mnt/disks/[MNT_DIR] 
   - sudo mount -o discard,defaults /dev/[DEVICE_ID] /mnt/disks/[MNT_DIR]
   - You will be able to see you partition running: df -h
5. chmod
- sudo chmod a+w /mnt/disks/[MNT_DIR]
6. Add autmatic mount operation so that you do not need to do it everytime you start the machine. This is done via the fstab file configurtion. <br> In a natshell, /dev/fstab is a dedicated place for the OS to mount disks. <br>To add disk you need to list it in Fstab file configuration. The structure of the file is a following: [Device] [Mount Point] [File System Type] [Options] [Dump] [Pass].
<br> There are two optinals to add it:
    1. By hand:
        1. Backup existing fstab file:   
            - sudo cp /etc/fstab /etc/fstab.backup
        2. Use blkid to filnd UUID of the persistent disk: 
            - sudo blkid /dev/[DEVICE_ID]
        3. Open /etc/fstab and add your device to the list:
        - UUID=[UUID_VALUE] /mnt/disks/[MNT_DIR] ext4 discard,defaults,[NOFAIL_OPTION] 0 2
        <br> Where [UUID_VALUE] is the number that was printed and [NOFAIL_OPTION] should be set to nofail (Meaning what happes to the OS boot when it can't mount. 
    2. Via single command (Replace sdb with the name you gave the disk):
        - echo UUID=`sudo blkid -s UUID -o value /dev/sdb` /mnt/disks/disk-1 ext4 discard,defaults,nofail 0 2 | sudo tee -a /etc/fstab

Once you remove the disk, remove the mount line
    

### Resizing disk of Instance

- Resize the disk size through Instance->Edit->Disk->Resize or by intialization instance from Snapshot but editting the size of the disk
- Run:
    - sudo lsblk 
        - To see all available disks 
        - Note that 'Mount on = /' means the root OS
    - df -h #To see which disks were mounted. You can see which process 
        - Shows the accessible devices/paritions. You can see you the mount happen successfully
- __If your disk has parition__ then you need to use growpart to resize the partiiton size:
    - sudo growpart /dev/sda 1
    - Shell Output: CHANGED: partition=1 start=2048 old: size=209713119 end=209715167 new: size=1048573919,end=1048575967
    - Now see the change in: sudo lsblk
- Now extend file system on the disk:
    - If you had partition run:
        - sudo resize2fs /dev/[DEVICE_ID][PARTITION_NUMBER]  #Example: sudo resize2fs /dev/sda1
    - If you didn't have parition run:
        - sudo resize2fs /dev/[DEVICE_ID]
    - Run df -h to see the change

## Buckets and Cloud Storage
Comparing diffrent [Storage Options ](https://cloud.google.com/compute/docs/disks/).

### Bucket Operators with gsutil
[Source](https://cloud.google.com/storage/docs/quickstart-gsutil#create)
- Create new bucket 
    - gsutil mb -l us-east1 gs://my-unique-bucket-name/
- Transfer data:
    - Copy files:
        - gsutil cp Desktop/kitten.png gs://my-awesome-bucket
    - Download file:
        - gsutil cp gs://my-awesome-bucket/kitten.png Desktop/kitten2.png
    - Creat folder and copy file to it:
        - gsutil cp gs://my-awesome-bucket/kitten.png gs://my-awesome-bucket/just-a-folder/kitten3.png
    - List objects in file on the bucket:
        - gsutil ls gs://my-awesome-bucket
- Permissions:
    -  grant all users read permission for the object stored in your bucket:
        - gsutil acl ch -u AllUsers:R gs://my-awesome-bucket/kitten.png
    - Remove permission:
        - gsutil acl ch -d AllUsers gs://my-awesome-bucket/kitten.png
    - Give a specific email address permission to read and write objects in your bucket:
        - gsutil iam ch user:jane@gmail.com:objectCreator,objectViewer gs://my-awesome-bucket
- Deleteing:
    - Delete object:
        - gsutil rm gs://my-awesome-bucket/kitten.png
     - Delete bucket:
         - gsutil rm -r gs://my-awesome-bucket
         
### Mounting Bucket onto Instance
The methdo to do this is by [Cloud Storage FUSE](https://cloud.google.com/storage/docs/gcs-fuse#using_feat_name) 
1. Create a bucket (Either through gsutil mb or through GUI
- Note - Bucket GUI are accessible at Storage->Browser ([link](https://console.cloud.google.com/storage/browser?_ga=2.11938484.-1862925903.1532348966&_gac=1.51962843.1533671701.CjwKCAjwhqXbBRAREiwAucoo-88kZ0DUOmhycGWVghcjrRnoEm2KZ32uM6mNtcrBmoA8iwtRKvQEIBoCsGcQAvD_BwE&project=manuel-dl&folder&organizationId))
2. Install [gcsfuse](https://github.com/GoogleCloudPlatform/gcsfuse/blob/master/docs/installing.md)
   - You might need to add credentials. Check out this [source](https://cloud.google.com/docs/authentication/production#auth-cloud-explicit-python)
3. mkdir /path/to/mount
4. gcsfuse --implicit-dirs my-bucket /path/to/mount
    - To see all the files in the directory run: gcsfuse __--implicit-dirs__ gaze_data_tzahi bucket 
    

Umount:<br>
fusermount -u /home/shared/local_folder/

## Reusing the same storage

### Reuse Disk
Unfortunetly, there is no way of using the same disk for multiple instances. YOu can read about it in [disk flexibility](https://cloud.google.com/compute/docs/disks/#flexibility) or in this [thread](https://stackoverflow.com/questions/26910960/share-a-persistent-disk-between-google-compute-engine-vms/26914885). According to the thread, there

You can however use the disk in a RO mode for multiple instance. [This guide](https://cloud.google.com/compute/docs/disks/add-persistent-disk#use_multi_instances) show how to attach a disk to an instance in RO moe:
1. Detach the disk from any R/W instance
2. Run attach command:
    - gcloud compute instances attach-disk [INSTANCE_NAME] --disk [DISK_NAME] --mode ro
3. Connect to the instance and __mount__ the disk

### Reuse Bucket

Bucket are the way to go with R/W storage connected to multiple disks. [Bucket](https://cloud.google.com/compute/docs/disks/gcs-buckets#mount_bucket) is what we explianed before. Assentially, you can just connect multiple instacnes to the same Bucket with no issues. It is just slower then disk